# Implementation of a Deep Neural Network
### Note:-
- Number of Hidden Layers = 2
- Output Layer size = 6
- This NN was deigned to recognize Numbers from 0 to 5 illustrated by using Fingers
- Optimization Function used is AdamOptimizer
- NN is trained using minibatches
- To evaluate performance, Train and Test sets are required
date : March 29,2020
author: m.mahyar_ali

In [1]:
#importing all Libraries
import tensorflow as tf
import numpy as np


**Creating Place Holder for X and Y so that we can change them at test time. Changing is required when using mini batches.**

In [12]:
def create_placeholders(n_x,n_y):
    X = tf.placeholder(dtype=tf.float32,shape=[n_x,None],name = 'X')
    Y = tf.placeholder(dtype=tf.float32,shape=[n_y,None],name = 'Y')
    return X,Y



**Initialize all the weights and biases. Xavier Initializer is used to overcome Gradient Exploding**

In [13]:
def initialize_parameters(n_x):
    tf.set_random_seed(1)
    parameters = {}
    parameters['W1'] = tf.get_variable('W1',[25,n_x],initializer = tf.contrib.layers.xavier_initializer(seed=1))
    parameters['b1'] = tf.get_variable('b1',[25,1],initializer=tf.zeros_initializer())
    parameters['W2'] = tf.get_variable('W2',[12,25],initializer= tf.contrib.layers.xavier_initializer(seed=1))
    parameters['b2'] = tf.get_variable('b2',[12,1],initializer = tf.zeros_initializer())
    parameters['W3'] = tf.get_variable('W3',[6,12],initializer = tf.contrib.layers.xavier_initializer(seed=1))
    parameters['b3'] = tf.get_variable('b3',[6,1],initializer = tf.zeros_initializer())
    
    return parameters

**Feed forward in the NN to compute Z3.Note:A3 is not required to compute because the cost function automatically computes it** <br>
**Required : X=Dataset , parameters = Dictionary of Weights and Biases**

In [14]:
def forward_propagation(X,parameters):
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    W3 = parameters['W3']
    b3 = parameters['b3']
    
    Z1 = tf.add(tf.matmul(W1,X),b1)
    A1 = tf.nn.relu(Z1)
    Z2 = tf.add(tf.matmul(W2,A1),b2)
    A2 = tf.nn.relu(Z2)
    Z3 = tf.add(tf.matmul(W3,A2),b3)
    
    return Z3

**To compute the cost. For this Example softmax Classifier is used **

In [26]:
def Cost(Z3,Y):
    logits = tf.transpose(Z3)
    labels = tf.transpose(Y)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=labels))
    
    return cost

**To Create Random Mini Batches of the dataset**

In [35]:
def random_mini_batches(X_train,Y_train,minibatch_size,seed=0):
    np.random.seed(seed) 
    m = X_train.shape[1]
    random_perm = list(np.random.permutation(m))
    X_train_S = X_train[:,random_perm]
    Y_train_S = Y_train[:,random_perm]
    num_minibatches = m//minibatch_size
    minibatches = []
    for batch in range(num_minibatches):
        minibatch_X = X_train_S[:,minibatch_size*batch:minibatch_size*(batch+1)]
        minibatch_Y = Y_train_S[:,minibatch_size*batch:minibatch_size*(batch+1)]
        minibatches.append((minibatch_X,minibatch_Y))
        
    if m%minibatch_size != 0 :
        minibatch_X = X_train_S[:,minibatch_size*num_minibatches:]
        minibatch_Y = Y_train_S[:,minibatch_size*num_minibatches:]
        minibatches.append((minibatch_X,minibatch_Y))
        
    return minibatches

**Putting all the pieces in one place. Initializing all the variables and then running the optimizer**

In [52]:
def model(X_train,Y_train,X_test,Y_test,learning_rate=0.0001,num_epochs = 1500,minibatch_size=32,print_cost=True):
    ops.reset_default_graph()
    seed = 3  
    n_x = X_train.shape[0]
    n_y = Y_train.shape[0]
    m = X_train.shape[1]
    X,Y = create_placeholders(n_x,n_y)
    
    parameters = initialize_parameters(n_x)
    
    Z3 = forward_propagation(X,parameters)
    
    cost = Cost(Z3,Y)
    
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    
    init = tf.global_variables_initializer()
    
    with tf.Session() as sess:
        sess.run(init)
        for epoch in range(num_epochs):
            epoch_cost = 0
            num_minibatches = int(m/minibatch_size)
            seed=seed+1
            minibatches = random_mini_batches(X_train,Y_train,minibatch_size,seed)
            for minibatch in minibatches:
                (minibatch_X, minibatch_Y) = minibatch
                _,minibatch_cost = sess.run([optimizer,cost],feed_dict = {X:minibatch_X,Y:minibatch_Y})
                
                epoch_cost += minibatch_cost/minibatch_size
            if (epoch%100==0):
                print("Cost after epoch "+str(epoch)+" is "+ str(epoch_cost))
    
        parameters = sess.run(parameters)
            
    